In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

# 设置参数
num_classes = 10  # CIFAR-10有10个类别
input_shape = (32, 32, 3)  # CIFAR-10图像的输入形状

# 加载CIFAR-10数据集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0  # 归一化到[0, 1]
x_test = x_test.astype('float32') / 255.0

# 转换标签为分类格式
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 定义Inception模块
def inception_module(x, filters):
    # Filters: (f1, f3_r, f3, f5_r, f5, f_pool)
    f1, f3_r, f3, f5_r, f5, f_pool = filters

    # 1x1卷积
    conv1 = layers.Conv2D(f1, (1, 1), padding='same', activation='relu')(x)

    # 1x1卷积后接3x3卷积
    conv3 = layers.Conv2D(f3_r, (1, 1), padding='same', activation='relu')(x)
    conv3 = layers.Conv2D(f3, (3, 3), padding='same', activation='relu')(conv3)

    # 1x1卷积后接5x5卷积
    conv5 = layers.Conv2D(f5_r, (1, 1), padding='same', activation='relu')(x)
    conv5 = layers.Conv2D(f5, (5, 5), padding='same', activation='relu')(conv5)

    # 3x3最大池化后接1x1卷积
    pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = layers.Conv2D(f_pool, (1, 1), padding='same', activation='relu')(pool)

    # 将所有分支连接在一起
    outputs = layers.concatenate([conv1, conv3, conv5, pool], axis=-1)
    return outputs

# 构建GoogLeNet模型
def build_googlenet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # 第一层
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # 第二层
    x = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = layers.Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Inception模块
    x = inception_module(x, filters=(64, 128, 128, 32, 32, 32))
    x = inception_module(x, filters=(128, 192, 192, 96, 96, 64))

    # 最大池化
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # 再次添加Inception模块
    x = inception_module(x, filters=(192, 96, 208, 16, 48, 64))
    x = inception_module(x, filters=(160, 112, 224, 24, 64, 64))
    x = inception_module(x, filters=(128, 128, 256, 24, 64, 64))
    x = inception_module(x, filters=(112, 144, 288, 32, 64, 64))
    x = inception_module(x, filters=(256, 160, 320, 32, 128, 128))

    # 全局平均池化和输出层
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model

# 构建GoogLeNet模型
googlenet_model = build_googlenet(input_shape, num_classes)

# 编译模型
googlenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
googlenet_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# 评估模型
loss, accuracy = googlenet_model.evaluate(x_test, y_test)
print(f"测试损失: {loss:.4f}, 测试准确率: {accuracy:.4f}")


Epoch 1/10
 86/625 ━━━━━━━━━━━━━━━━━━━━ 55s 103ms/step - accuracy: 0.1605 - loss: 2.1888

KeyboardInterrupt: 